In [1]:
# IMPORTAZIONE LIBRERIE
import pandas as pd
import logging
from bertopic import BERTopic
import nltk
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.metrics import silhouette_score
from IPython.display import display

# SOPPRESSIONE LOGGING NLTK
logging.getLogger('nltk').setLevel(logging.ERROR)

# DOWNLOAD RISORSE NLTK
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# FUNZIONI DI LOGGING
def log_info(msg):
    print(f"[INFO] {msg}")

def log_error(msg):
    print(f"[ERROR] {msg}")


# TEST DEI PARAMETRI HDBSCAN
def test_hdbscan_params_with_silhouette(documents, embeddings):
    param_list = [
        {'min_cluster_size': 10, 'min_samples': 5},
        {'min_cluster_size': 15, 'min_samples': 10},
        {'min_cluster_size': 20, 'min_samples': 15},
        {'min_cluster_size': 30, 'min_samples': 20},
        {'min_cluster_size': 40, 'min_samples': 25},
    ]

    best_score = -1
    best_params = None

    for params in param_list:
        clusterer = HDBSCAN(min_cluster_size=params['min_cluster_size'],
                            min_samples=params['min_samples'],
                            metric='euclidean', cluster_selection_method='eom')
        cluster_labels = clusterer.fit_predict(embeddings)

        n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
        n_outliers = sum(cluster_labels == -1)

        score = silhouette_score(embeddings, cluster_labels) if n_clusters > 1 else -1
        log_info(f"[TEST PARAMS] min_cluster_size={params['min_cluster_size']}, min_samples={params['min_samples']} -> "
                 f"Topics: {n_clusters}, Outliers: {n_outliers}, Silhouette Score: {score:.4f}")

        if score > best_score:
            best_score = score
            best_params = params

    log_info(f"✓ Parametri ottimali scelti: {best_params} con silhouette score = {best_score:.4f}")
    return best_params

# FUNZIONE PRINCIPALE DI TOPIC MODELING
def run_topic_modeling_notebook():
    log_info("=== INIZIO TOPIC MODELING CON BERTopic ===")

    # Percorsi dei file
    csv_input = "../data/processed/preprocessed_data.csv"
    csv_output = "../data/processed/bertopic_output.csv"
    model_output = "../models/bertopic_model"

    try:
        df = pd.read_csv(csv_input)
        log_info(f"✓ Dataset caricato: {csv_input} | Righe: {df.shape[0]}")
    except Exception as e:
        log_error(f"✗ Errore caricamento CSV: {str(e)}")
        raise

    if 'cleaned_text' not in df.columns:
        log_error("✗ La colonna 'cleaned_text' non è presente nel dataset.")
        raise ValueError("Colonna 'cleaned_text' mancante")

    # Preprocessing testi
    documents = df['cleaned_text'].astype(str).tolist()

    log_info("=== Estrazione embeddings per ottimizzazione parametri HDBSCAN ===")
    embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = embedding_model.encode(documents, show_progress_bar=True)

    log_info("=== TEST PARAMETRI HDBSCAN (ottimizzazione con Silhouette Score) ===")
    best_params = test_hdbscan_params_with_silhouette(documents, embeddings)

    try:
        log_info("✓ Inizializzazione modello BERTopic con parametri scelti...")

        hdbscan_model = HDBSCAN(min_cluster_size=best_params['min_cluster_size'],
                                min_samples=best_params['min_samples'],
                                metric='euclidean', cluster_selection_method='eom')

        topic_model = BERTopic(
            language="english",
            min_topic_size=10,
            hdbscan_model=hdbscan_model,
            embedding_model=embedding_model,
            verbose=True,
        )

        topics, probs = topic_model.fit_transform(documents)
        log_info("✓ Modello BERTopic addestrato.")

        # Aggiunta risultati al DataFrame
        df['Topic'] = topics
        df['Probability'] = probs

        # Generazione etichette automatiche
        topic_label_map = {
            topic: topic_model.get_topic(topic)[0][0] if topic != -1 else "outlier"
            for topic in topic_model.get_topics().keys()
        }
        df['Topic_Label'] = df['Topic'].map(lambda t: topic_label_map.get(t, "outlier"))

        # Salvataggio CSV
        df.to_csv(csv_output, index=False)
        log_info(f"✓ File salvato con successo: {csv_output}")

        # Salvataggio modello
        topic_model.save(model_output)
        log_info(f"✓ Modello salvato in: {model_output}")

        # Analisi e stampa riepilogo
        topic_info = topic_model.get_topic_info()
        log_info("=== ANALISI RAPIDA DEI TOPIC ===")
        for index, row in topic_info.head(10).iterrows():
            topic_num = row['Topic']
            label = topic_label_map.get(topic_num, "outlier")
            print(f"\n[TOPIC {topic_num}] - Documenti: {row['Count']}")
            print(f"Etichetta: {label}")

        # Etichette complete
        print("\n=== TOPIC LABELS GENERATI ===")
        for topic, label in topic_label_map.items():
            print(f"Topic {topic}: {label}")

        # Visualizzazione grafici direttamente nel notebook
        log_info("✓ Visualizzazione grafici nel notebook...")
        display(topic_model.visualize_barchart(top_n_topics=10))
        display(topic_model.visualize_topics())

    except Exception as e:
        log_error(f"✗ Errore durante il topic modeling: {str(e)}")
        raise

# ESECUZIONE (SOLO SE ESEGUITO COME SCRIPT)
if __name__ == "__main__":
    run_topic_modeling_notebook()


/Users/irene.gaita/PycharmProjects/FinSent/eda_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/irene.gaita/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/irene.gaita/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/irene.gaita/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


[INFO] === INIZIO TOPIC MODELING CON BERTopic ===
[INFO] ✓ Dataset caricato: ../data/processed/preprocessed_data.csv | Righe: 4821
[INFO] === Estrazione embeddings per ottimizzazione parametri HDBSCAN ===


Batches: 100%|██████████| 151/151 [00:11<00:00, 13.22it/s]


[INFO] === TEST PARAMETRI HDBSCAN (ottimizzazione con Silhouette Score) ===
[INFO] [TEST PARAMS] min_cluster_size=10, min_samples=5 -> Topics: 2, Outliers: 2961, Silhouette Score: 0.0092
[INFO] [TEST PARAMS] min_cluster_size=15, min_samples=10 -> Topics: 3, Outliers: 4261, Silhouette Score: -0.0259
[INFO] [TEST PARAMS] min_cluster_size=20, min_samples=15 -> Topics: 2, Outliers: 4635, Silhouette Score: -0.0486
[INFO] [TEST PARAMS] min_cluster_size=30, min_samples=20 -> Topics: 0, Outliers: 4821, Silhouette Score: -1.0000


2025-05-27 10:31:25,181 - BERTopic - Embedding - Transforming documents to embeddings.


[INFO] [TEST PARAMS] min_cluster_size=40, min_samples=25 -> Topics: 0, Outliers: 4821, Silhouette Score: -1.0000
[INFO] ✓ Parametri ottimali scelti: {'min_cluster_size': 10, 'min_samples': 5} con silhouette score = 0.0092
[INFO] ✓ Inizializzazione modello BERTopic con parametri scelti...


Batches: 100%|██████████| 151/151 [00:09<00:00, 16.11it/s]
2025-05-27 10:31:34,601 - BERTopic - Embedding - Completed ✓
2025-05-27 10:31:34,602 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-27 10:31:47,451 - BERTopic - Dimensionality - Completed ✓
2025-05-27 10:31:47,452 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-27 10:31:47,536 - BERTopic - Cluster - Completed ✓
2025-05-27 10:31:47,542 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-27 10:31:47,614 - BERTopic - Representation - Completed ✓
2025-05-27 10:31:47,730 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


[INFO] ✓ Modello BERTopic addestrato.
[INFO] ✓ File salvato con successo: ../data/processed/bertopic_output.csv
[INFO] ✓ Modello salvato in: ../models/bertopic_model
[INFO] === ANALISI RAPIDA DEI TOPIC ===

[TOPIC -1] - Documenti: 1479
Etichetta: outlier

[TOPIC 0] - Documenti: 125
Etichetta: bank

[TOPIC 1] - Documenti: 90
Etichetta: num_percent

[TOPIC 2] - Documenti: 80
Etichetta: nokia

[TOPIC 3] - Documenti: 79
Etichetta: meat

[TOPIC 4] - Documenti: 71
Etichetta: power

[TOPIC 5] - Documenti: 66
Etichetta: crane

[TOPIC 6] - Documenti: 65
Etichetta: support

[TOPIC 7] - Documenti: 62
Etichetta: mln

[TOPIC 8] - Documenti: 58
Etichetta: mln

=== TOPIC LABELS GENERATI ===
Topic -1: outlier
Topic 0: bank
Topic 1: num_percent
Topic 2: nokia
Topic 3: meat
Topic 4: power
Topic 5: crane
Topic 6: support
Topic 7: mln
Topic 8: mln
Topic 9: finnair
Topic 10: value
Topic 11: sanoma
Topic 12: paper
Topic 13: sale
Topic 14: metal
Topic 15: ceo
Topic 16: cut
Topic 17: beer
Topic 18: disclosed


# Scelte effettuate


Il testo utilizzato per l'analisi dei topic è stato sottoposto alla pipeline di preprocessing.

Per individuare i parametri più efficaci del clustering, è stata definita una lista di combinazioni di `min_cluster_size` e `min_samples` da testare. Per ogni combinazione:

- È stato eseguito un clustering preliminare usando **HDBSCAN**
- È stato calcolato il **Silhouette Score** per valutarne la qualità
- È stato registrato il numero di **topic rilevati** e di **outlier**

Ad esempio, durante una delle run:

| min_cluster_size | min_samples | Topics | Outliers | Silhouette Score |
|------------------|-------------|--------|----------|------------------|
| 10               | 5           | 2      | 2740     | 0.0111           |
| 30               | 20          | 0      | 4817     | -1.0000          |

I parametri con il punteggio più alto (anche se basso) sono stati scelti automaticamente. Nel caso sopra, `min_cluster_size=10` e `min_samples=5` sono risultati i migliori.

> ⚠️ **Nota importante**: poiché HDBSCAN e l'intero processo includono componenti stocastici (come l’UMAP di BERTopic), i risultati **possono variare ad ogni esecuzione**, generando un diverso numero di topic, outlier e punteggi di silhouette anche con gli stessi parametri.

## Parametri finali del modello BERTopic

Una volta selezionati i parametri ottimali, è stato inizializzato BERTopic con:

- **min_topic_size=10**: per permettere l’identificazione anche di cluster di dimensioni ridotte
- **embedding_model**: `all-MiniLM-L6-v2`, un modello leggero ma efficace per rappresentare semanticamente i testi
- **hdbscan_model**: il modello configurato con i parametri migliori trovati

Il modello è stato quindi addestrato e ha prodotto un numero elevato di topic — ad esempio **125 topic finali** — a fronte di soli **2 cluster principali** emersi dai test HDBSCAN. Questo comportamento è **atteso** e **coerente** con la logica di BERTopic, che:

- Usa HDBSCAN solo come **inizializzazione**
- Applica poi ulteriori raffinamenti (es. **c-TF-IDF**, rappresentazione semantica) per derivare sotto-topic
- È influenzato dalla **composizione e distribuzione semantica** dei documenti

## Salvataggio e analisi dei risultati

- Il **DataFrame** è stato arricchito con:
  - Numero di topic
  - Probabilità di appartenenza
  - Etichette automatiche dei topic
- I risultati sono stati **salvati in CSV** e il modello **serializzato** per usi futuri
- È stata effettuata un'**analisi esplorativa** dei topic principali e visualizzazioni interattive sono state generate direttamente nel notebook


